In [ ]:
#!pip install --no-cache-dir transformers sentencepiece
!pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-rry814p1
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-rry814p1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2138726 sha256=38d40e703d45ebfc96d9c6acadcf7d18500d1d726c6ffb76014e57a2239e3352
  Stored in directory: /tmp/pip-ephem-wheel-cache-dpq2k18o/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1


In [ ]:
!pip install rouge_score

In [ ]:
! pip install datasets py7zr
from datasets import load_dataset

     |████████████████████████████████| 245kB 7.1MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 122kB 12.4MB/s 
     |████████████████████████████████| 245kB 12.2MB/s 
     |████████████████████████████████| 358kB 14.7MB/s 
     |████████████████████████████████| 122kB 21.7MB/s 
     |████████████████████████████████| 1.9MB 15.1MB/s 
     |████████████████████████████████| 2.2MB 51.2MB/s 


In [ ]:
cp /content/drive/MyDrive/DLProject/run_summarization.py .

In [ ]:
import json
train = load_dataset("samsum", split='train')
test = load_dataset("samsum", split='test')
val =  load_dataset("samsum", split='validation')

with open('src-train.json', 'w') as f:
    for item in train:
        f.write("%s\n" % json.dumps(item))

with open('src-val.json', 'w') as f:
    for item in val:
        f.write("%s\n" % json.dumps(item))

with open('src-test.json', 'w') as f:
    for item in test:
        f.write("%s\n" % json.dumps(item))

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


Reusing dataset samsum (/root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)
Reusing dataset samsum (/root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#lynx --dump https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/run_summarization.py

SyntaxError: ignored

In [ ]:
#cp -r /tmp/tst-summarization/ /content/drive/MyDrive/DLProject/t5_small

In [ ]:
!python /content/run_summarization.py \
    --model_name_or_path t5-small \
    --do_train \
    --do_eval \
    --train_file /content/src-train.json \
    --validation_file /content/src-val.json \
    --source_prefix "summarize: " \
    --output_dir /tmp/tst-summarization \
    --overwrite_output_dir \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate \
    --text_column dialogue \
    --summary_column summary \

2021-03-19 08:53:59.107647: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/19/2021 08:54:00 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/19/2021 08:54:00 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/tmp/tst-summarization', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, logging_dir

In [ ]:
#Dataset({features: ['id', 'dialogue', 'summary'], num_rows: 14732 })
import pandas as pd
import numpy as np
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
! pip install datasets py7zr
from datasets import load_dataset
train = load_dataset("samsum", split='train')

data1 = pd.DataFrame(train)
data1['dialogue_processed'] = data1['dialogue'].str.split("\r\n")

def clean_text(sentences):
  data = []
  for sentence in sentences:
    for sent in sentence:
      sent = re.sub('[,\.!?]', ' ', sent).lower()
      sent = re.sub(r'[a-z]*:', '', sent)
      sent = re.sub(r'[^ \w\.]', ' ', sent)
      data.append(str(sent))
  return data

# Load the regular expression library

# Remove punctuation

data = clean_text(data1.dialogue_processed.values.tolist())
#len(data)
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [" ".join([word for word in simple_preprocess(str(doc)) 
             if word not in stop_words]) for doc in texts]
#data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['baked cookies want',
 'sure',
 'bring tomorrow',
 'voting election',
 'liberals always',
 '',
 'great',
 'hi',
 'bad mood tbh going lots stuff ended procrastinating',
 'plan',
 'oh know uni stuff unfucking room',
 'maybe tomorrow move ass everything',
 'going defrost fridge instead shopping eat defrosted veggies',
 'stuff recommend pomodoro technique breaks chores',
 'really helps',
 'thanks maybe',
 'also like using post kaban style',
 'rachel think ove bella',
 'dont say anything else',
 'mean',
 'open fu ing door outside',
 'hey overheard rick say something',
 'know',
 'say',
 'talking phone someone',
 'know',
 'telling happy',
 'damn',
 'saying like roommate',
 'wow feel',
 'thought good rommate',
 'nice place',
 'true man',
 'used love living moved boyfriend',
 'know saying',
 '',
 'honestly bothering much talk',
 'see going',
 'want get kind confrontation though',
 'maybe let go',
 'see goes future',
 'choice sam',
 'would talk clear air',
 'hi anyone remember date got married'

In [ ]:
re.sub(r'[a-z]*:', '', "amanda: boom")

' boom'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['baked', 'cookies', 'want']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1)]


In [ ]:
from pprint import pprint
import gensim

# number of topics
num_topics = 25
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
lda_model.save("10_topics_lda")
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import joblib

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                       
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=5000,          
                            )

data_vectorized = vectorizer.fit_transform(data)

lda_model = LatentDirichletAllocation(n_components=20, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
#lda_output = lda_model.fit_transform(data_vectorized)
#joblib.dump(lda_model, "/content/drive/MyDrive/DLProject/lda_model")
lda_model = joblib.load("/content/drive/MyDrive/DLProject/lda_model")

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
test_corp = id2word.doc2bow(["""For Jaccard similarity run 
  1) cd question2
  2) python jaccard_similarity_ranking.py
  3) Example run:  Enter number of queries 1
  4) Enter the query with terms separated by space universe was a small condensed hot spot
  5) Do you want to optimize retrieval by first doing index retrieval then ranking: YES or NO? YES
For part 2 and 3 of question 2 (tf-idf and cosine similairity run:
python relevant_documents_using_tf_idf_score.py"""], ["""For Jaccard similarity run 
  1) cd question2
  2) python jaccard_similarity_ranking.py
  3) Example run:  Enter number of queries 1
  4) Enter the query with terms separated by space universe was a small condensed hot spot
  5) Do you want to optimize retrieval by first doing index retrieval then ranking: YES or NO? YES
For part 2 and 3 of question 2 (tf-idf and cosine similairity run:
python relevant_documents_using_tf_idf_score.py"""], ["""For Jaccard similarity run 
  1) cd question2
  2) python jaccard_similarity_ranking.py
  3) Example run:  Enter number of queries 1
  4) Enter the query with terms separated by space universe was a small condensed hot spot
  5) Do you want to optimize retrieval by first doing index retrieval then ranking: YES or NO? YES
For part 2 and 3 of question 2 (tf-idf and cosine similairity run:
python relevant_documents_using_tf_idf_score.py"""])
id2word.token2id["think"]
lda_model.get_term_topics(54)
lda_model.get_document_topics(test_corp)

([(25780, 1)], {})

In [ ]:
#lda_model1 = gensim.models.LdaMulticore()

In [ ]:
pip install git+https://github.com/bmabey/pyLDAvis.git

  Cloning https://github.com/bmabey/pyLDAvis.git to /tmp/pip-req-build-dfx1f75a
  Running command git clone -q https://github.com/bmabey/pyLDAvis.git /tmp/pip-req-build-dfx1f75a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 8.7MB/s 
     |████████████████████████████████| 15.3MB 195kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=08bf69d1c6e5b6b0516534b389c7cc4a1a6a11b6c3b1dd93c847839a1d3f8e6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-74awwa6k/wheels/95/d1/2e/99f433ceb6aa00920eec343459fbe2604fd87e9bd895597da2
Successfully built pyLDAvis
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incom

In [ ]:
mytext = ["I am having fun lol funny"]

np.argmax(lda_model.transform(vectorizer.transform(mytext)))

<1x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn as sklvis
pyLDAvis.enable_notebook()
#vis = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word)
sklvis.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
#vis

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  error = np.finfo(np.float).max
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
7      -10.878068   10.538728       1        1  6.260627
16     -60.884026  -75.591820       2        1  5.915315
12      -2.336119  -53.289074       3        1  5.696678
5      -61.385319  -12.914634       4        1  5.624800
17     -28.737743  126.296997       5        1  5.482399
9        1.066606 -123.552811       6        1  5.479171
0       45.140720   -7.947869       7        1  5.369259
15    -140.795975  -98.530548       8        1  5.326269
4       61.531681  -83.240562       9        1  4.977580
19     -72.563057 -140.396881      10        1  4.854811
6       15.984491   63.578186      11        1  4.644426
10    -160.851624   68.945145      12        1  4.572191
8       85.961830   51.922935      13        1  4.539255
18     -99.020836  109.936348      14        1  4.532917
1     -176.764130   -8.681684      15        1  4.530877
2     -117.189880  -37.420200      16        1  4.487616
13     108.829918  -22.946203      17        1  4.487312
3       47.588097  121.934814      18        1  4.463588
14    -106.467834   30.176889      19        1  4.383452
11     -50.613804   59.328842      20        1  4.371456, topic_info=               Term         Freq        Total Category  logprob  loglift
2392           just  6474.000000  6474.000000  Default  30.0000  30.0000
1360            don  6079.000000  6079.000000  Default  29.0000  29.0000
2454           know  7258.000000  7258.000000  Default  28.0000  28.0000
2575           like  5432.000000  5432.000000  Default  27.0000  27.0000
4465          think  4412.000000  4412.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
4657  unfortunately   164.366144   165.306152  Topic20  -4.7950   3.1244
2266       internet   162.727171   163.667179  Topic20  -4.8050   3.1243
4239         street   161.223509   162.163517  Topic20  -4.8143   3.1243
1001        concert   157.606609   158.546617  Topic20  -4.8370   3.1241
4880         winter   155.371909   156.311917  Topic20  -4.8512   3.1240

[632 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.994821         100
93        3  0.998367        able
98       10  0.994529  absolutely
114       2  0.995440     account
134       5  0.998804    actually
...     ...       ...         ...
4961      1  0.998136       years
4966     18  0.997764         yep
4968      6  0.999852         yes
4972      3  0.999012   yesterday
4991     12  0.995086         yup

[602 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 17, 13, 6, 18, 10, 1, 16, 5, 20, 7, 11, 9, 19, 2, 3, 14, 4, 15, 12])

In [ ]:
pip install bertopic[visualization]

     |████████████████████████████████| 40kB 5.4MB/s 
     |████████████████████████████████| 6.4MB 17.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/73/ef/8967d406f3f85018ceb5efab50431e901683188f1741ceb053efcab26c87/numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 13.2MB 249kB/s 
     |████████████████████████████████| 2.1MB 45.4MB/s 
     |████████████████████████████████| 1.2MB 47.4MB/s 
     |████████████████████████████████| 1.2MB 48.5MB/s 
     |████████████████████████████████| 3.3MB 36.5MB/s 
     |████████████████████████████████| 901kB 51.2MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311661 sha256=ee23c0d0afd5a7ed08144d310024c397e1204d882866fa

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import joblib

data_words = data_words[:40000]

# Prepare embeddings
sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens")
#embeddings = sentence_model.encode(data_words, show_progress_bar=False)

# Create topic model
model = BERTopic()
#topics, probabilities = model.fit_transform(data_words, embeddings)

#model.save("/content/drive/MyDrive/DLProject/save_models/bert_topic_sentence")
#joblib.dump(embeddings, "/content/drive/MyDrive/DLProject/save_models/distil_sentence_embeddings")
#joblib.dump(topics, "/content/drive/MyDrive/DLProject/save_models/distil_sentence_topics")
#joblib.dump(probabilities, "/content/drive/MyDrive/DLProject/save_models/distil_sentence_probs")

model = BERTopic().load("/content/drive/MyDrive/DLProject/save_models/bert_topic_sentence")
topics = joblib.load("/content/drive/MyDrive/DLProject/save_models/distil_sentence_topics")
probabilities = joblib.load("/content/drive/MyDrive/DLProject/save_models/distil_sentence_probs")

new_topics, new_probs = model.reduce_topics(data_words, topics, probabilities, nr_topics=30)
model.visualize_topics()

NameError: ignored

In [ ]:
model1 = BERTopic().load("/content/drive/MyDrive/DLProject/save_models/bert_topic_sentence")

model1.topic_names

{-1: '-1_liked_project_without_less',
 0: '0_cufflins_neckline_hold_arm',
 1: '1_fileothetr_omnomnomnom_array_winword',
 2: '2_fact_pretty_zzz_foot',
 3: '3_due_zzz_footage_ford',
 4: '4_geri_discussed_example_foolish',
 5: '5_matters_tie_shall_zzz',
 6: '6_mon_trite_zzz_footage',
 7: '7_seriously_whaaat_stressed_zzz',
 8: '8_kk_idk_ikr_kthxbai',
 9: '9_welcome_alert_reception_hows',
 10: '10_looked_zzz_forcing_forcefully',
 11: '11_nd_part_since_zzz',
 12: '12_user_busiest_peer_keeps',
 13: '13_landline_pink_pyjamas_hehehe',
 14: '14_mercy_compliment_zzz_forcing',
 15: '15_yup_yum_considering_asking',
 16: '16_prob_probs_zzz_foot',
 17: '17_thats_im_foot_forcing',
 18: '18_realistic_gate_animal_im',
 19: '19_bye_goodnight_tot_yayayaya',
 20: '20_bye_goodbye_tolerate_lena',
 21: '21_yep_shed_rate_chapter',
 22: '22_basically_set_kinda_footage',
 23: '23_hod_um_footage_forcing',
 24: '24_arrangement_deal_packaging_coke',
 25: '25_photofile_degress_moi_ce',
 26: '26_happened_leg_matter_s

In [ ]:
pip install simpletransformers

In [ ]:
! pip install bertopic[visualization]

     |████████████████████████████████| 40kB 5.5MB/s 
     |████████████████████████████████| 15.3MB 207kB/s 
     |████████████████████████████████| 13.2MB 163kB/s 
     |████████████████████████████████| 6.4MB 48.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Using cached https://files.pythonhosted.org/packages/d8/b2/57495b5309f09fa501866e225c84532d1fd89536ea62406b2181933fb418/transformers-4.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/99/e0808cb947ba10f575839c43e8fafc9cc44e4a7a2c8f79c60db48220a577/sentencepiece-0.1.95-cp37-cp37m-manylinux2014_x86_64.whl
     |████████████████████████████████| 1.2MB 53.6MB/s 
  Using cached https://files.pythonhosted.org/packages/75/ee/67241dc87f266093c533a2d4d3d69438e57d7a90abb216fa076e7d475d4a/sacremoses-0.0.45-py3-none-

In [ ]:
! pip install datasets py7zr

from datasets import load_dataset

import json
train = load_dataset("samsum", split='train')
test = load_dataset("samsum", split='test')
val =  load_dataset("samsum", split='validation')

#Dataset({features: ['id', 'dialogue', 'summary'], num_rows: 14732 })
import pandas as pd
import numpy as np
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import joblib
#train = load_dataset("samsum", split='train')

data1 = pd.DataFrame(train)
data1['dialogue_processed'] = data1['dialogue'].str.split("\r\n")
sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens")

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

bert_model = BERTopic().load("/content/drive/MyDrive/DLProject/save_models/bert_topic_sentence")
#topics = joblib.load("/content/drive/MyDrive/DLProject/save_models/distil_sentence_topics")

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [" ".join([word for word in simple_preprocess(str(sentence)) 
             if word not in stop_words]) for sentence in texts]

def clean_text(sentences):
  train_data = []  
  wo_topics = []
  for sentence in sentences:
    sent_data = []
    top_data = []
    for sent1 in sentence:
      sent = sent1
      sent = re.sub('[,\.!?]', ' ', sent).lower()
      sent = re.sub(r'[A-Za-z]*:', '', sent)
      sent = re.sub(r'[^ \w\.]', ' ', sent)
      sent_data.append(str(sent))
    sent_data = list(sent_to_words(sent_data))
    sent_data = remove_stopwords(sent_data)
    wo_topics.append(sent_data)
  wo_topics = [item for wo in wo_topics for item in wo]
  embeddings = sentence_model.encode(wo_topics, show_progress_bar=False)
  topics, probs = bert_model.transform(wo_topics, embeddings)
  #print(topics)
  top = 0
  for j in range(len(sentences)):
    top_data = []
    for i in range(len(sentences[j])):
      top_data.append(sentences[j][i] + "|" + str(topics[top]) + "|")
      top += 1
    train_data.append(" ".join(top_data))
  return train_data

# Load the regular expression library
#len([[1,2], [1,2]].flatten())
# Remove punctuation

train_data = clean_text(data1.dialogue_processed.values.tolist())
#joblib.dump(train_data, "/content/drive/MyDrive/DLProject/data/train_data_with_topics")
#train_data = joblib.load("/content/drive/MyDrive/DLProject/data/train_data_with_topics")
#data1.head()

     |████████████████████████████████| 225kB 20.5MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 112kB 47.2MB/s 
     |████████████████████████████████| 245kB 54.5MB/s 
     |████████████████████████████████| 2.2MB 50.7MB/s 
     |████████████████████████████████| 122kB 55.7MB/s 
     |████████████████████████████████| 358kB 60.0MB/s 
     |████████████████████████████████| 1.9MB 55.5MB/s 


Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


Reusing dataset samsum (/root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)
Reusing dataset samsum (/root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: ignored

In [ ]:
def sent_to_words(sentence):
    # deacc=True removes punctuations
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords_str(texts):
    return " ".join([word for word in texts.split(" ") 
             if word not in stop_words])


clean_train_data = []
for sent in train_data:
  sent = re.sub('[,\.!?]', ' ', sent).lower()
  #sent = re.sub(r'[A-Za-z]*:', '', sent)
  sent = re.sub(r'[^ \w\.|]', ' ', sent)
  #sent = list(sent_to_words(sent))
  sent = remove_stopwords_str(sent)
  clean_train_data.append(sent)
#re.sub(r'[A-Za-z]*:', "", "Greg: Please tell me a game title it's totally worth buying|-1| Martin: With no hesitation|-1| Martin: Witcher 3 Wild Hunt|248| Martin: Game's worth your time|-1| Greg: I heard so. If you say that it's worth it, then I'm buying it right away.|-1| Martin: Great! Can't wait to talk it over with you :P|-1|")
#clean_train_data

In [ ]:
data1 = pd.DataFrame(train)

train_df = pd.DataFrame()
train_df["input_text"] = train_data
train_df["prefix"] = "summary"
train_df["target_text"] = data1.summary.values.tolist()

eval_df = pd.DataFrame()
val_data = pd.DataFrame(val)

eval_df["input_text"] = clean_text(val_data['dialogue'].str.split("\r\n"))
eval_df["prefix"] = "summary"
eval_df["target_text"] = val_data.summary.values.tolist()

test_df = pd.DataFrame()
val_data = pd.DataFrame(test)

test_df["input_text"] = clean_text(val_data['dialogue'].str.split("\r\n"))
test_df["prefix"] = "summary"
test_df["target_text"] = val_data.summary.values.tolist()

joblib.dump(train_df, "/content/drive/MyDrive/DLProject/data/train_df_nopre")
joblib.dump(eval_df, "/content/drive/MyDrive/DLProject/data/eval_df_nopre")
joblib.dump(test_df, "/content/drive/MyDrive/DLProject/data/test_df_nopre")

['/content/drive/MyDrive/DLProject/data/test_df_nopre']

In [ ]:
import joblib
train_df = joblib.load("/content/drive/MyDrive/DLProject/data/train_df_nopre")
eval_df = joblib.load("/content/drive/MyDrive/DLProject/data/eval_df_nopre")
test_df = joblib.load("/content/drive/MyDrive/DLProject/data/test_df_nopre")
#train_df.to_csv("/content/dataset_simil_topics.csv")
#test_df_topics

In [ ]:
import pandas as pd
! pip install simpletransformers

from simpletransformers.t5 import T5Model

#train_df = pd.read_csv("data/train_df.tsv", sep="\t").astype(str)
#eval_df = pd.read_csv("data/eval_df.tsv", sep="\t").astype(str)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False,
    "num_train_epochs": 3,
    "wandb_project": "Question Generation with T5",
}

model = T5Model("t5", "t5-small", args=model_args)

model.train_model(train_df, eval_data=eval_df)
#!cp -r "outputs/best_model" "/content/drive/MyDrive/DLProject/save_models/t5_w_topic2"

     |████████████████████████████████| 215kB 27.3MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 2.1MB 47.4MB/s 
     |████████████████████████████████| 8.2MB 38.1MB/s 
     |████████████████████████████████| 122kB 53.0MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 133kB 55.9MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 163kB 57.3MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 112kB 58.7MB/s 
     |████████████████████████████████| 4.2MB 44.2MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 122kB 49.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=ceff07149eb739ed90bb95533a438857809dbfb93933f1ab345f18332a5adc12
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c374

In [ ]:
import joblib
#joblib.dump(model, "/content/drive/MyDrive/DLProject/save_models/t5_with_topic_full_nopre")
#model2 = joblib.load("/content/drive/MyDrive/DLProject/save_models/t5_with_topic_full_nopre")

model1 = T5Model("t5", "/content/drive/MyDrive/DLProject/save_models/t5_w_topic2")

#With topics and sentence similarity

In [ ]:
! pip install rouge
from rouge import Rouge

preds = model1.predict(test_df.input_text.values.tolist())
#scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)

rouge = Rouge()
rouge.get_scores(preds, test_df.target_text.values.tolist(), avg=True)

{'rouge-1': {'f': 0.3630502078241465,
  'p': 0.5161383515213515,
  'r': 0.30699986013962477},
 'rouge-2': {'f': 0.15487468836485996,
  'p': 0.22411607621152993,
  'r': 0.13121616778936726},
 'rouge-l': {'f': 0.3544162023122647,
  'p': 0.493742098875691,
  'r': 0.29872146781045916}}

In [ ]:
#Custom dataset

preds = model1.predict(clean_text([['Aseem: Wow! You got a new laptop! Which laptop is this?', 'Tharun: It is the new m1 macbook air.',  'Aseem: That\'s great! From where did you get it!', 'Tharun: From the Apple store near Kal katchi', 'Aseem: How is the battery life?', 'Tharun: It lasts for around 17 hours on a single charge']]))
preds

['Tharun got a new macbook air from the Apple store near Kal katchi.']

#Pure T5 without topics

In [ ]:
! pip install datasets py7zr

from datasets import load_dataset

import json
train = load_dataset("samsum", split='train')
test = load_dataset("samsum", split='test')
val =  load_dataset("samsum", split='validation')

#Dataset({features: ['id', 'dialogue', 'summary'], num_rows: 14732 })
import pandas as pd
import numpy as np
import joblib
#train = load_dataset("samsum", split='train')

data1 = pd.DataFrame(train)
  
train_df = pd.DataFrame()
train_df["input_text"] = data1.dialogue.values.tolist()
train_df["prefix"] = "summary"
train_df["target_text"] = data1.summary.values.tolist()

eval_df = pd.DataFrame()
val_data = pd.DataFrame(val)

eval_df["input_text"] = val_data.dialogue.values.tolist()
eval_df["prefix"] = "summary"
eval_df["target_text"] = val_data.summary.values.tolist()

test_df = pd.DataFrame()
val_data = pd.DataFrame(test)

test_df["input_text"] = val_data.dialogue.values.tolist()
test_df["prefix"] = "summary"
test_df["target_text"] = val_data.summary.values.tolist()

#joblib.dump(train_df, "/content/drive/MyDrive/DLProject/data/train_df_topics")
#joblib.dump(eval_df, "/content/drive/MyDrive/DLProject/data/eval_df_topics")


["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style",
 "Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside",
 "Sam: hey  overheard rick say something\r\nSam: i don't k

In [ ]:
import pandas as pd
! pip install simpletransformers

from simpletransformers.t5 import T5Model


#train_df = pd.read_csv("data/train_df.tsv", sep="\t").astype(str)
#eval_df = pd.read_csv("data/eval_df.tsv", sep="\t").astype(str)

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 1,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False,

    "wandb_project": "Question Generation with T5",
}

model_wo_topic = T5Model("t5", "t5-small", args=model_args)

model_wo_topic.train_model(train_df, eval_data=eval_df)
!cp -r "outputs/best_model" "/content/drive/MyDrive/DLProject/save_models/t5_wo_top"

In [ ]:
import joblib
#joblib.dump(model_wo_topic, "/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")
#model_wo_topic = joblib.load("/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")

model_wo_topic1 = T5Model("t5", "/content/drive/MyDrive/DLProject/save_models/t5_wo_top/best_model")


#With sentence similarity

In [ ]:
!pip install rouge
from rouge import Rouge

preds = model_wo_topic1.predict(test_df.input_text.values.tolist())
#scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)

clean_preds = []
clean_test_df = []

for i in range(len(preds)):
  if len(preds[i].strip()) != 0:
    clean_preds.append(preds[i])
    clean_test_df.append(test_df.target_text.values.tolist()[i])

rouge = Rouge()
rouge.get_scores(clean_preds, clean_test_df, avg=True)

{'rouge-1': {'f': 0.33158982568695683,
  'p': 0.4551665298397314,
  'r': 0.29120966624552735},
 'rouge-2': {'f': 0.13726872477084429,
  'p': 0.1838609233217077,
  'r': 0.12299557541903668},
 'rouge-l': {'f': 0.3299660180333304,
  'p': 0.4604773892355592,
  'r': 0.2822780220463189}}

#T5 with sentence similarity and topics

In [ ]:
! pip install bertopic[visualization]

     |████████████████████████████████| 40kB 4.7MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 6.4MB 46.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.2MB 245kB/s 
  Using cached https://files.pythonhosted.org/packages/73/ef/8967d406f3f85018ceb5efab50431e901683188f1741ceb053efcab26c87/numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 2.1MB 51.5MB/s 
     |████████████████████████████████| 1.2MB 48.7MB/s 
     |████████████████████████████████| 1.2MB 38.3MB/s 
     |████████████████████████████████| 901kB 44.8MB/s 
     |████████████████████████████████| 3.3MB 49.2MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311675 sha256=42bbf52b5a128c459da134291f9b542962f055589183eb6

In [ ]:
#! pip install datasets py7zr

#from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import joblib

import json
train = pd.read_csv("/content/new_final.csv",converters={"dialogs": lambda x: x.strip("[]").split(", ")})

#Dataset({features: ['id', 'dialogue', 'summary'], num_rows: 14732 })

#train = load_dataset("samsum", split='train')

#data1 = pd.DataFrame(train)
#data1['dialogue_processed'] = data1['dialogue'].str.split("\r\n")
sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens")

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

bert_model = BERTopic().load("/content/drive/MyDrive/DLProject/save_models/bert_topic_sentence")
#topics = joblib.load("/content/drive/MyDrive/DLProject/save_models/distil_sentence_topics")

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [" ".join([word for word in simple_preprocess(str(sentence)) 
             if word not in stop_words]) for sentence in texts]

def clean_text(sentences):
  train_data = []  
  wo_topics = []
  for sentence in sentences:
    sent_data = []
    top_data = []
    for sent1 in sentence:
      sent = sent1
      sent = re.sub('[,\.!?]', ' ', sent).lower()
      sent = re.sub(r'[A-Za-z]*:', '', sent)
      sent = re.sub(r'[^ \w\.]', ' ', sent)
      sent_data.append(str(sent))
    sent_data = list(sent_to_words(sent_data))
    sent_data = remove_stopwords(sent_data)
    wo_topics.append(sent_data)
  wo_topics = [item for wo in wo_topics for item in wo]
  embeddings = sentence_model.encode(wo_topics, show_progress_bar=False)
  topics, probs = bert_model.transform(wo_topics, embeddings)
  #print(topics)
  top = 0
  for j in range(len(sentences)):
    top_data = []
    for i in range(len(sentences[j])):
      top_sent = sentences[j][i] + "|" + str(topics[top]) + "|"
      top_sent = re.sub('[\'\",\.!?]', "", top_sent)
      top_data.append(top_sent)
      top += 1
    train_data.append(" ".join(top_data))
  return train_data

# Load the regular expression library
#len([[1,2], [1,2]].flatten())
# Remove punctuation

#train_data = clean_text(train.dialogs.values.tolist())
#joblib.dump(train_data, "/content/drive/MyDrive/DLProject/data/train_data_with_topics")
#train_data = joblib.load("/content/drive/MyDrive/DLProject/data/train_data_with_topics")
#data1.head()

In [ ]:
train_df = pd.DataFrame()
train_data = [re.sub('[\'\",\.!?]', "", " ".join(item)) for item in train.dialogs.values.tolist()]
#train_data = clean_text(data1['dialogue'].str.split("\r\n")[:7000])
#train = pd.DataFrame()
#train['summ'] = data1.summary.values.tolist()[:7000]
train_df["input_text"] = train_data[:5000]
train_df["prefix"] = "summary"
train_df["target_text"] = train.summ.values.tolist()[:5000]


eval_df = pd.DataFrame()
#val_data = pd.DataFrame(val)

eval_df["input_text"] = train_data[5000:6000]
eval_df["prefix"] = "summary"
eval_df["target_text"] = train.summ.values.tolist()[5000:6000]

test_df = pd.DataFrame()
#val_data = pd.DataFrame(test)

test_df["input_text"] = train_data[6000:]
test_df["prefix"] = "summary"
test_df["target_text"] = train.summ.values.tolist()[6000:]


In [ ]:
import pandas as pd
! pip install simpletransformers

from simpletransformers.t5 import T5Model


model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    #"max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 3,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "fp16": False,
    #"num_train_epochs": 3,
    #"wandb_project": "Question Generation with T5",
}

model_w_top_sent = T5Model("t5", "t5-small", args=model_args)

model_w_top_sent.train_model(train_df, eval_data=eval_df)
#!cp -r "outputs/best_model" "/content/drive/MyDrive/DLProject/save_models/t5_w_topics_07"

#!cp -r "outputs/best_model" "/content/drive/MyDrive/DLProject/save_models/t5_wo_topics_07"


Using Adafactor for T5


(1875,
 {'eval_loss': [2.451923385620117, 2.416381416320801, 2.441657551765442],
  'global_step': [625, 1250, 1875],
  'train_loss': [3.1615188121795654, 2.85040283203125, 1.7194401025772095]})

In [ ]:
import joblib
#joblib.dump(model_wo_topic, "/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")
#model_wo_topic = joblib.load("/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")

model_w_top_sent = T5Model("t5", "/content/drive/MyDrive/DLProject/save_models/t5_wo_topics_07")


#with topics

In [ ]:
!pip install rouge
from rouge import Rouge

preds = model_w_top_sent.predict(test_df.input_text.values.tolist())
#scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)

clean_preds = [];clean_test_df = []
for i in range(len(preds)):
  if len(preds[i].strip()) != 0:
    clean_preds.append(preds[i])
    clean_test_df.append(test_df.target_text.values.tolist()[i])

rouge = Rouge()
rouge.get_scores(clean_preds, clean_test_df, avg=True)

{'rouge-1': {'f': 0.35247208080425746,
  'p': 0.4899455118050239,
  'r': 0.3043528457883617},
 'rouge-2': {'f': 0.14905765485744898,
  'p': 0.21097564650947565,
  'r': 0.12977890679538995},
 'rouge-l': {'f': 0.3533212622192173,
  'p': 0.49283916802231753,
  'r': 0.30004367504271673}}

#With nothing, baseline

In [ ]:
import joblib
#joblib.dump(model_wo_topic, "/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")
#model_wo_topic = joblib.load("/content/drive/MyDrive/DLProject/save_models/t5_wo_topic")

model_w_nothing = T5Model("t5", "/content/drive/MyDrive/DLProject/save_models/t5_w_top_sent")


In [ ]:
!pip install rouge
from rouge import Rouge

preds = model_w_nothing.predict(test_df.input_text.values.tolist())
#scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)

clean_preds = []
clean_test_df = []

for i in range(len(preds)):
  if len(preds[i].strip()) != 0:
    clean_preds.append(preds[i])
    clean_test_df.append(test_df.target_text.values.tolist()[i])

rouge = Rouge()
rouge.get_scores(clean_preds, clean_test_df, avg=True)

{'rouge-1': {'f': 0.3379200092649441,
  'p': 0.48250394354143517,
  'r': 0.28696478977407275},
 'rouge-2': {'f': 0.13329889950439244,
  'p': 0.19430923765119004,
  'r': 0.11400609738901339},
 'rouge-l': {'f': 0.3313725458329725,
  'p': 0.46805575486624773,
  'r': 0.2782328469110652}}

In [ ]:
clean_preds

['Ryan is working on a night at the gym. Darren is going to see Ryan next',
 'Adele and Martin are watching Ozark. The house of cards season 6 is coming.',
 "Sue's printer is not working. She will turn it off and turn it on.",
 'Julianna is bored. Julianna is going to watch a book on healthy lifestyle',
 'Darrell and Heidi are going to Lublanska.',
 'Fred is trying to download the pdfs from his google account. Lana is going to send',
 'Damian is at the gym. He is surprised when he first saw Dwayne',
 'Robin is angry about the landlord paying his landlord. Robin will call her at 333.',
 'Della is in Berlin. Thomas is in Berlin. Della is in Berlin. Thomas is',
 'Mike and Ian are trying to repair their BMW. Mike has bought a new one.',
 'Adrian is angry at Ronaldo. Ronaldo is angry at Adrian.',
 'Harry is going out tomorrow night.',
 "Mary and Tom miss Tom's little wombat.",
 'Tommy is busy with his party friend. Tommy has a gym shopping with his dad drink with',
 'Caro is launching her o

In [ ]:
clean_test_df

["Darren didn't see Ryan at the gym as he's been working nights in the warehouse. Ryan isn't coming on Friday either. Ryan will give Darren a bell.",
 'Martin recommends a new movie called Money Heist to Adele as he thinks Ozark is too American. Both thinks that the season 5 of the House of Cards was great but Adele did not know that Kevin Spacey will not be part of the cast in season 6. ',
 "Sue's printer stopped working. Sam and John are trying to help but to no avail. Sam is going to repair it when he's back.",
 'Julianna is bored. Ivona suggests that they watch the new TV show "Deal or No Deal" together.',
 "Heidi came back. Darrell has moved to Lublanska, Heidi and Darrell work close to each other now, but Darrell has to commute 2,5h every day. Darrell doesn't want to take his car as it burns a lot of fuel. He doesn't have the money to buy a new car. Heidi and Darrell will meet for lunch at 12:30. ",
 "Fred can't access their google account and wants Lana to send him some pdfs.",


In [ ]:
test_df.input_text.values.tolist()

['Darren: Hi Buddy Didnt see you at the gym You OK|353| Ryan: Yeah mate Been working bloody nights too knackered to do much|-1| Darren: Nightmare You coming Friday|566| Ryan: Nah working again arent it Management dont give a shit about us lot in the warehouse|-1| Darren: Yeah but its double time aint it Youll be coining it in See you next week then|593| Ryan: Maybe I dunno if I can be arsed Ill give you a bell See you mate|-1|',
 'Adele: What was the name of the show you told me|-1| Martin: Money heist its on netflix|302| Adele: You liked it|-1| Martin: a lot youll like it too :) its in spanish|192| Adele: OK Ill check it out tonight|291| Martin: Im watching Ozark right now but after money heist it seems so i dont know too american :) |-1| Adele: house of cards season 6 is coming|-1| Martin: I know s5 was great s6 is going to be odd without spacey|-1| Adele: What do you mean|42| Martin: hes not in the cast|237| Adele: Really|7| Martin: yes RIP Frank Underwood :(|294|',
 'Sue: My printe